In [1]:
#ipython magic
%reset -f
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
#general imports
import sys
sys.path.append('..')
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import time

In [3]:
#notebook specific imports
from utils.artifact_removal import artifact_removal
from utils.utils_funcs import threshold_detect, stim_start_frame, paq_data
import tifffile
from utils.paq2py import paq_read
import copy


In [4]:
#global plotting params
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
sns.set()
sns.set_style('white')

In [5]:
tiff_path = r'/home/jamesrowland/Documents/packerstation/rlees/Data/2019-04-25/RL036/2019-04-25_RL036_t-001/2019-04-25_RL036_t-001_Cycle00001_Ch3.tif'
naparm_path = r'/home/jamesrowland/Documents/packerstation/rlees/Data/2019-04-25/RL036/2019-04-25_RL036_NAPARM'
paq_path = r'/home/jamesrowland/Documents/packerstation/rlees/Data/2019-04-25/RL036/2019-04-25_RL036_t-001.paq'

In [6]:
stack = tifffile.imread(tiff_path)
stack = stack[0:500,:,:]
stack_original = copy.deepcopy(stack)
stack.shape

(500, 512, 512)

In [7]:
paq = paq_read(paq_path)
stim_frames = stim_start_frame(paq, 'markpoints2packio')
frame_clock = paq_data(paq, 'frame_clock', threshold_ttl=True)
stim_frames

[1088735,
 1108819,
 1128902,
 1148988,
 1169072,
 1189155,
 1209241,
 1228655,
 1248739,
 1268825,
 1288908,
 1308992,
 1329075,
 1349161,
 1369245,
 1388659,
 1408745,
 1428829,
 1448912,
 1468998,
 1489082,
 1509165,
 1529249,
 1548665,
 1568749,
 1588832,
 1608919,
 1629002,
 1649086,
 1669169,
 1689256,
 1708669,
 1728753,
 1748839,
 1768923,
 1789006,
 1809090,
 1829176,
 1849260,
 1868674,
 1888757,
 1908843,
 1928927,
 1949011,
 1969097,
 1989181,
 2009264,
 2028678,
 2048764,
 2068848,
 2088932,
 2109015,
 2129101,
 2149185,
 2169269,
 2188683,
 2208769,
 2228853,
 2248936,
 2269020,
 2289106,
 2309190,
 2329273,
 2348687,
 2368774,
 2388857,
 2408941,
 2429025,
 2449111,
 2469194,
 2489278,
 2508692,
 2528778,
 2548862,
 2568946,
 2589029,
 2609116,
 2629199,
 2649283,
 2668697,
 2688783,
 2708867,
 2728951,
 2749034,
 2769121,
 2789204,
 2809288,
 2828702,
 2848788,
 2868872,
 2888956,
 2909039,
 2929126,
 2949209,
 2969293,
 2988707,
 3008791,
 3028877,
 3048961,
 3069044,


In [303]:
duration_ms = 250 #length of stim
duration_samples = (duration_ms / 1000) * paq['rate']

In [304]:
#use this to remove frames during stim based on paq
to_remove = []

for stim_frame in stim_frames: 

    # frame indexes that fall during the stim
    in_stim = np.where((frame_clock >= stim_frame) & (frame_clock <= stim_frame + duration_samples))[0]
    
    #empircal observation, these are the frames with artifact
    in_stim = np.append(in_stim, in_stim[-1]+1)
    in_stim = np.append(in_stim, in_stim[-1]+1)
    
    to_remove.append(in_stim)

to_remove = np.ravel(to_remove)

In [305]:
# stack[to_remove, :, :] = 0
# tifffile.imwrite('test.tiff', stack)

In [306]:
t1 = time.time()

to_remove = [i for i in to_remove if i < stack.shape[0]]

stack_removed = artifact_removal(stack, remove_me=to_remove, width_thresh=3)

t2 = time.time()

t2-t1

59.67378783226013

In [307]:

tifffile.imwrite('test_removed.tiff', stack_removed)
tifffile.imwrite('test.tiff', stack_original)

In [308]:
len(to_remove)

130

In [309]:
to_remove

[115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 474,
 475,
 476,
 477,
 478,
 479,
 480,
 481,
 482,
 483]